In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
sample_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
# SexとEmbarkedをダミー変数化する
train = pd.get_dummies(train, columns = ['Sex', 'Embarked'])
test = pd.get_dummies(test, columns = ['Sex', 'Embarked'])

In [4]:
# 使わなそうと判断した列を絡むごと削除する
# inplaceは初期値がFalseで新しいDataFlameが返される。もとのDataFlameを変更したかったらTrueにする
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1, inplace = True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis = 1, inplace = True)

In [5]:
X= train.drop(['Survived'], axis=1)
y = train['Survived']

lightgbmを使っていく

In [6]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

In [8]:
gbm = lgb.LGBMClassifier(objective='binary')

In [9]:
gbm.fit(X_train, y_train, eval_set = [(X_test, y_test)], 
       early_stopping_rounds = 20, 
       verbose = 10);

Training until validation scores don't improve for 20 rounds
[10]	valid_0's binary_logloss: 0.487687
[20]	valid_0's binary_logloss: 0.452407
[30]	valid_0's binary_logloss: 0.449514
[40]	valid_0's binary_logloss: 0.453922
Early stopping, best iteration is:
[24]	valid_0's binary_logloss: 0.447878


In [10]:
pred = gbm.predict(X_test, num_iteration = gbm.best_iteration_)
print('score', round(accuracy_score(y_test, pred)* 100,2), '%')

score 80.6 %


In [11]:
test_pred = gbm.predict(test, num_iteration = gbm.best_iteration_)
sample_submission['Survived'] = test_pred
sample_submission.to_csv('submission_lgbm', index = False)

# KFOLDで分割してやってみる